# Imports

In [63]:
import pandas as pd

pd.options.mode.chained_assignment = None
pd.options.display.max_rows = 500
pd.options.display.max_seq_items = 500

import os
import re
from tqdm.notebook import tqdm
import xmltodict
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic

Constants and functions

In [64]:
load_dotenv("/Volumes/1TB Home SSD/GitHub/.env")

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [65]:
## OpenAI ---------------------------------------------------------------
# GPT-4 has a 128k tokens context window
# Max output ist 4096 tokens

openai_client = OpenAI()
OPENAI_SYSTEM_MESSAGE = """"You are a helpful assistant."""
TEMPERATURE = 0.0


def call_openai(prompt, modelId="gpt-4o", temperature=TEMPERATURE, max_tokens=4096):
    """Call OpenAI's API with a prompt and return the completion."""
    try:
        completion = openai_client.chat.completions.create(
            model=modelId,
            temperature=temperature,
            max_tokens=max_tokens,
            messages=[
                {"role": "system", "content": OPENAI_SYSTEM_MESSAGE},
                {"role": "user", "content": prompt},
            ],
        )
        return completion.choices[0].message.content

    except Exception as e:
        print(f"Error: {e}")
        return None


## Anthropic ---------------------------------------------------------------
# All models have a 200k token context window
# Max output ist 4096 tokens
anthropic_client = Anthropic(api_key=ANTHROPIC_API_KEY)

HAIKU = "claude-3-haiku-20240307"
OPUS = "claude-3-opus-20240229"
SONNET = "claude-3-5-sonnet-20240620"


def call_anthropic(prompt, modelId=HAIKU, temperature=TEMPERATURE, max_tokens=4096):
    try:
        message = anthropic_client.messages.create(
            model=modelId,
            max_tokens=max_tokens,
            temperature=temperature,
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
        )
        return message.content[0].text

    except Exception as e:
        print(f"Error: {e}")
        return None

In [107]:
BASE_PROMPT = """Du erhältst einen Projektsteckbrief eines KI-Projekts in der Schweizer Verwaltung. Du sollst daraus die wichtigsten Informationen extrahieren.

Hier der Projektsteckbrief:
<steckbrief>
{}
</steckbrief>

Extrahiere folgende Merkmale:
* Projektname
* Sprache
* Links
* Einsetzende Einrichtungen
* Themenfelder
* Startdatum
* Enddatum
* Projektstatus
* Projektleitung
* Ansprechperson
* Datentyp
* Typ von Machinelearning («Kompontenten des ML»)

Extrahiere zudem die Projektbeschreibung als sehr kurze Zusammenfassung von maximal 2 Sätzen.
Wenn im Steckbrief für ein Merkmal keine Angaben genannt sind, schreibe «Keine Angabe».

Gib die Ergebnisse im XML-Format aus. Beginne direkt mit dem XML, kommentiere nichts.

Das Format des XML-Outputs ist wie folgt:

<?xml version="1.0" encoding="UTF-8"?>
<projekt>
  <projektname>Maschinelle Übersetzung Bund</projektname>
  <sprache>Deutsch, Französisch</sprache>
  <links>https://www.deepl.com/translator</links>
  <einsetzende_einrichtungen>Gesamte Bundesverwaltung</einsetzende_einrichtungen>
  <themenfelder>Maschinelle Übersetzung, Textredaktionshilfe</themenfelder>
  <startdatum>Januar 2019</startdatum>
  <enddatum>30. Juni 2024</enddatum>
  <projektstatus>In Betrieb</projektstatus>
  <projektleitung>Kompetenzzentrum Sprachtechnologien, BK</projektleitung>
  <ansprechperson>Monika Röthlisberger, Stéphanie di Rosa</ansprechperson>
  <datentyp>Nicht klassifizierte Textschnipsel, Word-, Powerpoint- und pdf-Dateien; keine personenbezogenen Daten</datentyp>
  <ml_komponenten>DeepL verwendet nur öffentlich verfügbare Texte des Bundes für maschinelles Lernen.</ml_komponenten>
  <projektbeschreibung>Mitarbeitende der Bundesverwaltung müssen häufig fremdsprachige Texte lesen oder verfassen. DeepL Pro, ein generisches maschinelles Übersetzungssystem, wird eingesetzt, um rund 1 Mio. Standardseiten pro Monat zu übersetzen und die Sprachdienste zu entlasten.</projektbeschreibung>
</projekt>

Die Inhalte der XML-Tags sind immer reine Textstrings, NIEMALS Dictionaries. Wenn du ein Dictionary erhältst, musst du es in einen String umformulieren. Beispiel: {{'person': ['Christopher Carlen', 'Camilo Chiang']}} wird zu 'Christopher Carlen, Camilo Chiang'."""

# Parse list with LLMs

Preprocessing
- Download PDF
- Open in Acrobat and export as Word DOCX
- Convert DOCX to Markdown with pandoc:
    - `pandoc -f docx -t markdown -o _input/CNAI_Projekte_D_8_0.md _input/CNAI_Projekte_D_8_0.docx` 

Read Markdown and split into projects

In [104]:
file_path = "_input/CNAI_Projekte_D_8_0.md"

with open(file_path, "r") as file:
    text = file.read()

# Split the text into individual project descriptions.
data = text.split("> []{#_bookmark")
df = pd.DataFrame(data, columns=["text"])

df = df[df["text"] != ""]
df["word_count"] = df["text"].apply(lambda x: len(x.split()))

# Remove splits that only contain a department name.
df = df[df["word_count"] > 11]

df.text = df.text.apply(lambda x: x.strip())
df.reset_index(drop=True, inplace=True)

display(df.shape[0])
display(df.head())

67

,text,word_count
0,1 .anchor}PoC Departementszuteilung\n\n+------...,308
1,3 .anchor}KD-Chatbot\n\n+---------------------...,742
2,4 .anchor}Mailbot\n\n+-----------------------+...,702
3,6 .anchor}ADELE-System (Entwurf zur Überarbeit...,436
4,7 .anchor}Camvis\n\n+-----------------------+-...,336


Parse data with Claude v3

In [110]:
results = []
for text in tqdm(df["text"]):
    prompt = BASE_PROMPT.format(text)
    result = call_anthropic(prompt, temperature=0.0, max_tokens=4096)
    results.append(result)

  0%|          | 0/67 [00:00<?, ?it/s]

In [ ]:
rows = []
for result in results:
    xml_data = result.split('<?xml version="1.0" encoding="UTF-8"?>')[1].strip()
    result_dict = xmltodict.parse(xml_data)
    row = pd.DataFrame(result_dict).T.reset_index(drop=True)
    rows.append(row)
    
df_results = pd.concat(rows)
df_results.reset_index(drop=True, inplace=True)

df_results.replace("Keine Angabe", None, inplace=True)
df_results.replace("", None, inplace=True)
df_results.replace("-", None, inplace=True)

df_results.to_excel("_output/CNAI_Projekte_D_8_0.xlsx", index=False)
df_results.to_parquet("_output/CNAI_Projekte_D_8_0.parq")

In [116]:
display(df_results.info())
display(df_results.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   projektname                67 non-null     object
 1   sprache                    67 non-null     object
 2   links                      37 non-null     object
 3   einsetzende_einrichtungen  67 non-null     object
 4   themenfelder               58 non-null     object
 5   startdatum                 67 non-null     object
 6   enddatum                   62 non-null     object
 7   projektstatus              67 non-null     object
 8   projektleitung             66 non-null     object
 9   ansprechperson             64 non-null     object
 10  datentyp                   52 non-null     object
 11  ml_komponenten             52 non-null     object
 12  projektbeschreibung        57 non-null     object
dtypes: object(13)
memory usage: 6.9+ KB


None

,projektname,sprache,links,einsetzende_einrichtungen,themenfelder,startdatum,enddatum,projektstatus,projektleitung,ansprechperson,datentyp,ml_komponenten,projektbeschreibung
0,PoC Departementszuteilung,Deutsch und Französisch,None,Sektion Bundesratsgeschäfte der Bundeskanzlei,"Texterkennung, Kategorisierung",10.3.2022,13.6.2022,"Proof of Concept, Projekt abgeschlossen, Machb...",Sektion Digitale Dienste BK,"Gautschi Michael, Luggen Michael",Text (strukturiert),Überwachtes Lernen,Parlamentarische Vorstösse müssen für die Bean...
1,KD-Chatbot,Französisch,None,Konsularische Direktion EDA,Texterkennung,2019,2021,"Prototyp, Projekt abgeschlossen",Konsularische Direktion EDA,Kato Yuri,Strukturierte Daten,Überwachtes Lernen,"Der Chatbot wurde entwickelt, um die Informati..."
2,Mailbot,Deutsch,None,Informatik EDA,Texterkennung,2018,laufend,Produktion,Informatik EDA,Tomaso Bezzola,Strukturierte Daten,Überwachtes Lernen,Der IT Helpdesk bearbeitet mit maximal 14 Mita...
3,ADELE-System (Entwurf zur Überarbeitung der Me...,Französisch,https://www.experimental.bfs.admin.ch/expstat/...,"Bundesamt für Statistik, Raum und Umwelt, Bode...","Bilderkennung, räumliche Kartierung, Veränderu...",2018,laufend,Produktion,BFS/RU/GEO/AREA,"Claudio Facchinetti, Gillian Milani","Bild (unstrukturiert), Sekundärdaten (struktur...","Überwachtes Lernen, Deep Learning",Erkennung von Veränderungen und Klassifizierun...
4,Camvis,Englisch,None,MeteoSchweiz,Bilderkennung,2020,laufend,Prototyp,"EDI, MeteoSchweiz",Christian Sigg,Unstrukturierte Daten,Überwachtes Lernen,Die automatische Schätzung der meteorologische...
